In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import pyaudio
import wave 
from pydub import AudioSegment
from pydub.playback import play

D:\ACCS\Voice_commands\custom\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [8]:
def record(command):
    # Record in chunks of 1024 samples
    chunk = 1024
    # 16 bits per sample
    sample_format = pyaudio.paInt16
    channels = 1
    sample_rate = 16000
    seconds = 1.5
    filename = "{}.wav".format(command)

    # Create an interface to PortAudio
    pa = pyaudio.PyAudio()
    stream = pa.open(format = sample_format, channels = channels,
                    rate = sample_rate, input = True, frames_per_buffer = chunk)
    print("Recording...")
    # Initialize array that be used for storing frames
    frames = [] 

    for i in range(0, int(sample_rate / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    # Terminate - PortAudio interface
    pa.terminate()

    print("Done !!!")

    # Save the recorded data in a .wav format
    sf = wave.open(filename, 'wb')
    sf.setnchannels(channels)
    sf.setsampwidth(pa.get_sample_size(sample_format))
    sf.setframerate(sample_rate)
    sf.writeframes(b''.join(frames))
    sf.close()
    return filename

def decode_audio(audio_binary):
    audio, _ = tf.audio.decode_wav(contents = audio_binary)
    return tf.squeeze(audio, axis = -1)

def preprocess(filepath):
    audio_binary = tf.io.read_file(filepath)
    waveform = decode_audio(audio_binary)
    waveform = tf.cast(waveform, dtype = tf.float32)
    spectrogram = tf.signal.stft(waveform, frame_length = 255, frame_step = 188)
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram[..., tf.newaxis]
    return spectrogram

In [3]:
commands = ['Down', 'Engine', 'Off', 'On', 'One', 'Three', 'Two', 'Up', 'Window', 'Wiper']

In [4]:
model = tf.keras.models.load_model(os.path.join("trained_weights", "model1_12_27.h5"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 32, 32, 1)         0         
                                                                 
 normalization (Normalizatio  (None, 32, 32, 1)        3         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0

In [42]:
while True:
    l = input("Do you want to continue (y/n) : ")
    if l == 'n':
        break
    file = record("test")
    spec = preprocess(file)
    y = model.predict(spec.numpy().reshape(1, 124, 129, 1))
    print(commands[np.argmax(y, axis = 1)[0]])

Do you want to continue (y/n) :  


Recording...
Done !!!
Engine


Do you want to continue (y/n) :  


Recording...
Done !!!
On


Do you want to continue (y/n) :  


...

Do you want to continue (y/n) :  


Recording...
Done !!!
One


Do you want to continue (y/n) :  


Recording...
Done !!!
Engine


Do you want to continue (y/n) :  n
